In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Q1.1
Download the Adult dataset. Clean up the dataset and create x_train, y_train, x_test, y_test for training feature, training value, test feature, test label. All of these variables should be numpy arrays. Provide summary statistics for your training and test datasets so that TA can verify the correctness of your procedure.

In [2]:
# Load training data and testing data 
column_list = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'label']
train_df = pd.read_csv('adult.data', names=column_list)
test_df = pd.read_csv('adult.test', names=column_list, skiprows=1)
original_features_list = list(test_df.columns)

In [3]:
# remove all rows with missing values. 
train_df.dropna(axis=0, how='any', inplace=True)
test_df.dropna(axis=0, how='any', inplace=True)
for col in column_list:
    train_df = train_df[train_df[col] != ' ?']
    test_df = test_df[test_df[col] != ' ?']

# label value will be 1 for '>50K' and 0 otherwise
train_df.replace(regex = {'>50K': 1, '<=50K' : 0}, inplace=True)
test_df.replace(regex = {'>50K': 1, '<=50K' : 0}, inplace=True)

# features with discrete-values be converted to "1-of-K" encoding
dummy_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
# Include a particular feature value only if this unique value appears more than ten times in the training data.
train_v = train_df[dummy_columns]
train_df = train_df[train_v.replace(train_v.apply(pd.Series.value_counts)).gt(10).all(1)]
test_v = test_df[dummy_columns]
test_df = test_df[test_v.replace(train_v.apply(pd.Series.value_counts)).gt(10).all(1)]

train_df = pd.get_dummies(train_df, columns=dummy_columns)
test_df = pd.get_dummies(test_df, columns=dummy_columns)
new_features_list = list((train_df.drop(['label'], axis=1)).columns)

print(train_df.shape)
print(test_df.shape)


/Users/michelle/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


(30152, 103)
(15055, 103)


In [4]:
#summary statistic
train_df.describe(include='all')

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,label,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
count,30152.000000,3.015200e+04,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,...,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000,30152.000000
mean,38.440568,1.897916e+05,10.121319,1092.370025,88.266085,40.931348,0.248972,0.030976,0.068553,0.739089,...,0.001128,0.003615,0.000365,0.002355,0.001393,0.000564,0.000597,0.911880,0.002123,0.000531
std,13.135362,1.056567e+05,2.550204,7407.547899,404.046306,11.979776,0.432425,0.173257,0.252696,0.439139,...,0.033562,0.060017,0.019097,0.048469,0.037297,0.023738,0.024426,0.283474,0.046023,0.023030
min,17.000000,1.376900e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.176248e+05,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,37.000000,1.784250e+05,10.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,47.000000,2.376255e+05,13.000000,0.000000,0.000000,45.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
# summary statistic 
test_df.describe(include='all')

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,label,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,...,native-country_ Portugal,native-country_ Puerto-Rico,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia
count,15055.000000,1.505500e+04,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,...,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000,15055.000000
mean,38.769711,1.896046e+05,10.112255,1120.188907,89.071471,40.950714,0.245566,0.030422,0.068615,0.732049,...,0.001860,0.004384,0.000598,0.001993,0.000864,0.000797,0.000531,0.915510,0.001262,0.000465
std,13.381046,1.056274e+05,2.558629,7704.274825,406.347469,12.064465,0.430437,0.171751,0.252807,0.442907,...,0.043087,0.066068,0.024444,0.044597,0.029374,0.028222,0.023046,0.278131,0.035504,0.021559
min,17.000000,1.349200e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.166450e+05,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,37.000000,1.779510e+05,10.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,48.000000,2.385810e+05,13.000000,0.000000,0.000000,45.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,3770.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
# convert into numpy array
Y_train = train_df['label'].to_numpy()
Y_test = test_df['label'].to_numpy()
X_train = train_df.drop(['label'], axis=1).to_numpy()
X_test = test_df.drop(['label'], axis=1).to_numpy()

# Q1.2
Derive the gradient and hessian matrix for the new E(w).

## Gradient matrix for E(w)

assume that $z_1 = tln\sigma(w^T\phi)$, $z_2 = (1-t)ln[1-\sigma(w^T\phi)]$

$\nabla E(w) = \Lambda w + \frac{\partial z_1}{\partial w} + \frac{\partial z_2}{\partial w}$

$\frac{\partial z_1}{\partial w} = t[1-\phi(w^T\phi)\phi]$

$\frac{\partial z_2}{\partial w} = (t-1)\sigma(w^T\phi)\phi$

$\frac{\partial z}{\partial w} = \frac{\partial z_1}{\partial w} + \frac{\partial z_2}{\partial w} = (t-\sigma(w^T\phi))\phi$

So $\nabla E(w)$ can be derived as $\Lambda w - (t-\sigma(w^T\phi) = \Lambda w + \phi^T(y-t)$

## Hessian matrix for E(w)

The gradient matrix for $E(w)$ is $\nabla E(w) = \Lambda w + \phi^T(y-t)$, and $\frac{\partial z}{\partial w} = \phi^T(y-t) =  \sum _{n=1}^{N} (y_n-t_n)\phi_n$.

The Hessian matrix for $z$ can be derived as $\sum _{n=1}^{N} y_n(1-y_n)\phi_n \phi_n^T = \Phi^T R \Phi$, which $R = y_n(1-y_n)$.

So the $\nabla \nabla E(x)$ can be derived as $\Lambda I + \Phi^T R \Phi$

In [9]:
# Closed form solution for ridge regression
def closed_form_reg_solution(X, t, b): 
    n, m = X.shape
    I = np.eye((m))
    return (np.linalg.inv(X.T @ X + b * I) @ X.T @ t)

# Y sigmoid function
def Y_sigmoid_func(w, X):
    return 1 / (1 + np.exp(-w.T @ X.T))

# gradient matrix of E(w)
def gradient_Ew_func(w, lamda, X, t, Y):
    return lamda @ w + (X.T @ (Y-t))

# hessian matrix of E(w)
def hassian_Ew_func(X, Y, lamda):
    n, m = X.shape
    I_n = np.eye((n))
    I_m = np.eye((m))
    R = (Y * (1-Y)).T * I_n
    return X.T @ R @ X + (lamda @ I_m)

# E(w) error function
def Ew_func(w, Y, t, lamda):
    return 0.5 * w.T @ lamda @ w - t @ np.log(Y) + (1- t @ (np.log(1 - Y)))

# new w
def new_weight_func(w, X, lamda, t):
    Y = Y_sigmoid_func(w, X)
    gradient_Ew = gradient_Ew_func(w, lamda, X, t, Y)
    hassian_Ew = hassian_Ew_func(X, Y, lamda)
    w_new = w - np.linalg.inv(hassian_Ew) @ gradient_Ew
    return w_new

In [10]:
X = X_train
t = Y_train
b = 1
lamda = np.eye((X.shape[1]))

w = closed_form_reg_solution(X, t, b)
w_new = new_weight_func(w, X, lamda, t)

# Q1.3
Create your mylogistic_l2 class. Show the learned 𝑤 as well as test accuracy for the cases below. If 𝑤 is too long for you, show selected 𝑤 for continuous-valued, binary-valued, and the constant term.

In [26]:
class mylogistic_l2():
    def __init__(self, reg_vec, max_iter, tol, add_intercept):
        self.reg_vec = reg_vec
        self.max_iter = max_iter
        self.tol = tol
        self.add_intercept = add_intercept
    
    def fit(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train
    
    def get_weights(self):
        if self.add_intercept:
            intercept = np.ones((self.X_train.shape[0], 1))
            self.X_train = np.hstack((self.X_train, intercept))
        
        n, m = X_train.shape
        I_n = np.eye((n))
        I_m = np.eye((m))
        
        lamda_total = 0
        for i in range(self.reg_vec.shape[0]):
            lamda_total += self.reg_vec[i][i]
        b = lamda_total / self.reg_vec.shape[0]
        
        weights = closed_form_reg_solution(self.X_train, self.Y_train, b)      
        for step in range(self.max_iter):
            Y = Y_sigmoid_func(weights, self.X_train)
            
            # calculate old weight error
            old_error = Ew_func(weights, Y, self.Y_train, self.reg_vec)
            weights = new_weight_func(weights, self.X_train, self.reg_vec, self.Y_train)
            new_error = Ew_func(weights, Y, self.Y_train, self.reg_vec)
            
            #tol = abs(new_error - old_error) / old_error
            tol = abs(new_error - old_error)
            
            if tol < self.tol:
                break
        return weights
    
    def predict(self, X_test):
        if self.add_intercept:
            intercept = np.ones((X_test.shape[0], 1))
            X_test = np.hstack((X_test, intercept))
        weights = self.get_weights()
        #print(weights)
        Y = weights.T @ X_test.T
        return np.where(Y < 0.5, 0, 1)

def accuracy_func(ypred, Y_test):
    correct = 0
    length = ypred.shape[0]
    for i in range(length):
        if ypred[i] == Y_test[i]:
            correct += 1
    return correct / length

## Case 1 
lambda = 1 for all coefficients

In [13]:
lambda_vec1 = np.eye((X_train.shape[1]))

logic1 = mylogistic_l2(reg_vec = lambda_vec1, max_iter = 1000, tol = 1e-5, add_intercept = False)
logic1.fit(X_train, Y_train)

In [14]:
ypred_1 = logic1.predict(X_test)
accuracy_func(ypred_1, Y_test)

[ 2.41796566e-02  7.82707987e-07  1.08400372e-01  3.27463928e-04
  6.53736607e-04  2.87834834e-02 -9.76323532e-02 -8.03641945e-01
 -6.01053744e-01 -4.80847877e-01 -1.08487923e+00 -9.26905393e-01
 -1.08986922e+00 -7.90571297e-01 -9.11950199e-01 -5.68761340e-01
 -6.73786320e-01 -7.05811615e-01 -1.26228488e+00 -8.62519888e-01
 -2.24963151e-01 -1.71046728e-01  2.62067023e-01  9.22538483e-01
 -4.23305027e-01  5.22291321e-01 -9.58284273e-01  9.73254244e-01
 -2.11696115e-01 -1.28689242e+00  1.03199860e+00  4.53675532e-01
 -1.14890618e+00 -1.80071377e+00 -1.25888819e+00 -1.07510332e+00
 -2.66283185e-01 -2.35635588e-01  5.28024283e-01 -1.23731090e+00
 -9.89154176e-01 -5.66780806e-01 -1.14193057e+00 -1.78279101e+00
  2.42865945e-01  3.05633246e-01  2.78021462e-02  4.30173678e-01
 -3.99442820e-01 -7.98477717e-01 -7.16337180e-01 -1.35346836e+00
 -1.82462710e+00 -8.94856329e-01  5.02936922e-01 -1.28483941e+00
 -7.15535839e-01 -1.04896648e+00 -1.14020777e+00 -8.95280255e-01
 -2.96597444e+00 -2.11885

0.8418465626037861

## Case 2
lambda = 1 for all but the intercept, no regularization for intercept term.

In [15]:
lambda_vec2 = np.eye((X_train.shape[1]+1))

logic2 = mylogistic_l2(reg_vec = lambda_vec2, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic2.fit(X_train, Y_train)
ypred_2 = logic2.predict(X_test)

[ 2.45446275e-02  7.98282217e-07  1.67756907e-01  3.27738403e-04
  6.54331881e-04  2.90881427e-02  1.99256461e-01 -5.06096798e-01
 -3.00440222e-01 -1.85659222e-01 -7.87953042e-01 -6.30096032e-01
 -9.56341943e-01 -4.97867988e-01 -6.75858090e-01 -3.94972376e-01
 -2.30340427e-01 -2.74589294e-01 -8.62126708e-01 -5.20861517e-01
 -2.76481320e-01 -1.64246632e-01  1.51433614e-01  6.40531797e-01
 -2.99144069e-01  3.52792214e-01 -7.17679155e-01  7.48089987e-01
 -1.46010834e-01 -1.00841211e+00  1.26871756e+00  7.59444064e-01
 -8.81298895e-01 -1.51819614e+00 -9.84334351e-01 -8.03250919e-01
 -1.13265116e-01 -8.19790035e-02  6.79518510e-01 -1.08769344e+00
 -8.36301526e-01 -4.11922763e-01 -9.87562799e-01 -1.69486500e+00
  3.93865848e-01  4.58291164e-01  1.79115384e-01  5.82023328e-01
 -2.46555383e-01 -4.91873716e-01 -3.85008462e-01 -1.04170801e+00
 -1.49916238e+00 -5.62758486e-01  8.13180258e-01 -9.10717464e-01
 -3.19372946e-01 -6.58777065e-01 -7.74795583e-01 -5.03667739e-01
 -2.00734362e+00 -1.15998

In [16]:
accuracy_func(ypred_2, Y_test)

0.8419129857190302

## Case 3
lambda = 1 for numerical-valued features, lambda = 0.5 for binary-valued features, no regularization for intercept term.

In [17]:
lambda3_list = []
for col in new_features_list:
    # lambda = 1 for numerical-valued features
    if col in original_features_list:
        lambda3_list.append(1)
    else:
        lambda3_list.append(0.5)

lambda3_list.append(0)
lambda_vec3 = np.diag(np.array(lambda3_list))

logic3 = mylogistic_l2(reg_vec = lambda_vec3, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic3.fit(X_train, Y_train)
ypred_3 = logic3.predict(X_test)

[ 2.51943584e-02  8.25079205e-07  2.73123474e-01  3.28484653e-04
  6.55896509e-04  2.96242121e-02  7.49673973e-01  4.14197651e-02
  2.53152824e-01  3.60260306e-01 -2.40375315e-01 -8.42326581e-02
 -1.07989890e+00  2.48623967e-02 -2.54945992e-01 -8.49103770e-02
  5.97334592e-01  5.19136868e-01 -1.50775661e-01  9.48662461e-02
 -3.68894364e-01 -1.50016575e-01 -4.26772078e-02  1.45384620e-01
 -7.54976301e-02  5.54830533e-02 -6.39419836e-01  3.56934143e-01
 -2.68642754e-02 -5.96666351e-01  1.88844897e+00  1.28796019e+00
 -4.95569350e-01 -1.09875499e+00 -5.82320923e-01 -4.03097552e-01
  1.80015776e-01  2.12404103e-01  9.71987214e-01 -8.08700111e-01
 -5.48903334e-01 -1.15233855e-01 -6.98705781e-01 -2.03017272e+00
  6.84579948e-01  7.56105130e-01  4.70939024e-01  8.78937484e-01
  4.67471251e-02 -2.56247513e-02  1.94024147e-01 -5.33943211e-01
 -9.49276327e-01  2.00222635e-02  1.29479788e+00 -3.04493145e-01
  3.54270212e-01 -1.77715865e-02 -1.71000238e-01  1.38994758e-01
 -4.26014303e-01  4.26014

In [18]:
accuracy_func(ypred_3, Y_test)

0.8418465626037861

# Q1.4 
Further split the training data into subtraining (90%) and tuning (10%) to search for the best hyperparameters. Set the regularization coefficient for the constant term to zero. Allow different regularizations for continuous-valued and binary-valued features. Let 𝑎1 and 𝑎2 denote the regularization coefficients for continuous-valued and binary-valued features. Search the best 𝑎1 and 𝑎2 and report the test accuracy using the best hyper-parameters. You should follow the procedure to search for the best hyperparameters.

In [19]:
# split the training data into subtraining and tunning
X_train, X_tuning, Y_train, Y_tunning = train_test_split(X_train, Y_train, test_size=0.1)

# Choose a set of grids among a reasonable range.
grids = [0.01, 0.05, 0.1, 0.5, 1, 5, 10, 20, 80, 100]

In [27]:
# Conduct grid search with the constraint that 𝑎1=𝑎2
best_accuracy = 0
for g in grids:
    lambda_list = []
    for col in new_features_list:
        lambda_list.append(g)
    lambda_list.append(0)
    lambda_vec_tunning1 = np.diag(np.array(lambda_list))
    logic_tunning1 = mylogistic_l2(reg_vec = lambda_vec_tunning1, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic_tunning1.fit(X_train, Y_train)
    ypred_tunning1 = logic_tunning1.predict(X_tuning)
    accuracy = accuracy_func(ypred_tunning1, Y_tunning)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_a = g

In [29]:
# Fix  𝑎1=𝑎∗1, and search 𝑎2 for the best value, call the result the new 𝑎∗2
best_accuracy = 0
a1 = best_a
a2_star = 0

for g in grids:
    lambda_list = []
    for col in new_features_list:
        # a1 for numerical-valued features
        if col in original_features_list:
            lambda_list.append(a1)
        else:
            lambda_list.append(g)
    lambda_list.append(0)
    lambda_vec_tunning2 = np.diag(np.array(lambda_list))
    
    logic_tunning2 = mylogistic_l2(reg_vec = lambda_vec_tunning2, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic_tunning2.fit(X_train, Y_train)
    ypred_tunning2 = logic_tunning2.predict(X_tuning)
    accuracy = accuracy_func(ypred_tunning1, Y_tunning)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        a2_star = g

In [30]:
# Fix  𝑎2=𝑎∗2, and search 𝑎1 for the best value. Report the selected 𝑎1 and  𝑎2.
best_accuracy = 0
a1_star = 0

for g in grids:
    lambda_list = []
    for col in new_features_list:
        # a1 for numerical-valued features
        if col in original_features_list:
            lambda_list.append(g)
        else:
            lambda_list.append(a2_star)
    lambda_list.append(0)
    lambda_vec_tunning3 = np.diag(np.array(lambda_list))
    
    logic_tunning3 = mylogistic_l2(reg_vec = lambda_vec_tunning3, max_iter = 1000, tol = 1e-5, add_intercept = True)
    logic_tunning3.fit(X_train, Y_train)
    ypred_tunning3 = logic_tunning3.predict(X_tuning)
    accuracy = accuracy_func(ypred_tunning3, Y_tunning)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        a1_star = g

In [31]:
# Train a model using the selected hyper-parameters, and report the test accuracy.
lambda_list = []
for col in new_features_list:
    # a1 for numerical-valued features
    if col in original_features_list:
        lambda_list.append(a1_star)
    else:
        lambda_list.append(a2_star)
lambda_list.append(0)
lambda_vec_tunning4 = np.diag(np.array(lambda_list))

logic_tunning4 = mylogistic_l2(reg_vec = lambda_vec_tunning4, max_iter = 1000, tol = 1e-5, add_intercept = True)
logic_tunning4.fit(X_train, Y_train)
ypred_tunning4 = logic_tunning4.predict(X_test)
accuracy = accuracy_func(ypred_tunning4, Y_test)
print(accuracy)

0.8410494852208569


In [32]:
print(a1_star)
print(a2_star)

0.01
0.01


# Q1.5 
Use sklearn.linear_model.LogisticRegression to train and test the model (including hyperparameter tuning). Compare the estimated parameters and test accuracy with those from your own models.

In [25]:
best_accuracy = 0
best_g = 0
for g in grids:
    logic_tunning5 = LogisticRegression(C = g, tol = 1e-5, max_iter = 1000)
    logic_tunning5.fit(X_train, Y_train)
    accuracy = logic_tunning5.score(X_tuning, Y_tuning)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_g = g

logic_tunning5_final = LogisticRegression(C = best_g, tol = 1e-5, max_iter = 1000)
logic_tunning5_final.fit(X_train, Y_train)
accuracy_final = logic_tunning5.score(X_test, Y_test)
print('estimated parameters: ', best_g, ', accuracy: ', accuracy_final)

/Users/michelle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8347392892726669
0.8160743938890734


/Users/michelle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/michelle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.844968449020259
0.8176021255396878


/Users/michelle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/michelle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


...

0.844902025905015


/Users/michelle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8439056791763534
0.8187313184988376


/Users/michelle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/michelle/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8180670873463965
estimated parameters:  0.1 , accuracy:  0.844968449020259


# Discussion

sklearn.linear_model.LogisticRegression 做完 hyperparameter tuning 後 predict 出的最佳 accuracy 為 0.844968449020259、regularization 為 0.1。

而 mylogistic_l2 做完 hyperparameter tuning 後 predict 出的 accuracy 為 0.8410494852208569，在此 training data 中 tun 出的 continuous-valued features regularization 為 0.01、binary-valued features regularization 同樣為 0.01。

雖然 sklearn.linear_model.LogisticRegression predict 出的 accuracy 比 mylogistic_l2 分開 continuous-valued features regularization 與 binary-valued features regularization 時 predict 出的 accuracy 高，但是同樣用 mylogistic_l2 進行 predict 時可發現，分開 regularization 的 accuracy 比兩種 features 使用同一 regularization 時高，並且加上 intercept 後也可看出 accuracy (0.8419129857190302) 比沒有加 intercept (0.8418465626037861) 來得高。